In [37]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [90]:
from time import sleep, time_ns

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import pandas as pd


In [29]:
"""
The code below for chrome browser in linux.
path = "/usr/bin/chromedriver"
option = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=path, options=option)
"""
url = "http://115.124.110.196:8080/epeek/"
browser = webdriver.Firefox()
browser.get(url)
browser.implicitly_wait(2) #wait for two second to load the website fully

#Select the value from the season dropdown
season_drop = Select(browser.find_element(By.ID, 'seasonOpt'))
season_drop.select_by_value('उन्हाळी')
sleep(1)

#Select the value from the division dropdown
div_drop = Select(browser.find_element(By.ID, 'divOpt'))
div_drop.select_by_value('amravati')
sleep(1)

#Select the value from the district dropdown
dist_drop = Select(browser.find_element(By.ID, 'distOpt'))
dist_drop.select_by_value('-')
sleep(1)

#Select the values from the village dropdown
village_drop = Select(browser.find_element(By.ID, 'villOpt'))
village_drop.select_by_value('-')

#Select button
button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/table/tbody/tr[11]/td/button')
button.click()
sleep(20)

object_content = browser.find_element(By.TAG_NAME, 'object')
browser.switch_to.frame(object_content)

rows = len(browser.find_elements(By.CSS_SELECTOR, 'tr'))
print(rows)
row = 2
data_list = []
while(row<rows-1):
    data = {}
    try:
        data['sl_no'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[1]').text
        data['village'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[2]').text
        data['survey_account_number'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[3]').text
        data['peak_monitoring_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[4]').text
        data['permanent_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[5]').text
        data['current_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[6]').text
        data['total_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[7]').text
        
        data_list.append(data)
        row += 1

    except NoSuchElementException:
        break

print(data_list)

data_set = pd.DataFrame(data_list)
data_set.to_csv(f'data/data_{time_ns()}.csv')


browser.close()



8
[{'sl_no': '1', 'village': 'अकोला', 'survey_account_number': '4069', 'peak_monitoring_area': '5956.97', 'permanent_area': '0.00', 'current_area': '29.76', 'total_area': '5986.73'}, {'sl_no': '2', 'village': 'अमरावती', 'survey_account_number': '3616', 'peak_monitoring_area': '5386.93', 'permanent_area': '0.00', 'current_area': '14.59', 'total_area': '5401.52'}, {'sl_no': '3', 'village': 'बुलडाणा', 'survey_account_number': '9050', 'peak_monitoring_area': '12503.17', 'permanent_area': '0.00', 'current_area': '35.94', 'total_area': '12539.11'}, {'sl_no': '4', 'village': 'यवतमाळ', 'survey_account_number': '3968', 'peak_monitoring_area': '6788.61', 'permanent_area': '0.00', 'current_area': '47.67', 'total_area': '6836.28'}, {'sl_no': '5', 'village': 'वाशिम', 'survey_account_number': '1364', 'peak_monitoring_area': '2125.23', 'permanent_area': '0.00', 'current_area': '16.05', 'total_area': '2141.29'}]


In [3]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [1]:
#Imports required packages
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep

In [2]:
# URL = "http://115.124.110.196:8080/epeek/rptViewDist.jsp?ccode=-&blk=-&dist=-&div=amravati&season=खरीप&divName=अमरावती&distName=सर्व&blkName=सर्व&ccodeName=सर्व&seasonName=खरीप"
base_url = "http://115.124.110.196:8080/epeek/"
# ccode
village_code = "272100040267930000"
# blk
block_code = "4"
# dist
district_code = "36"
# div
en_division_name = "kokan"
# season
mr_season_name = "खरीप"
# divName
mr_division_name = "कोंकण"
# distName
mr_district_name = "पालघर"
# blkName
mr_block_name = "जव्हार"
# ccodeName
mr_village_name = "अनंतनगर"

url_params = ''

if en_division_name == '-':
   url_params =  f"rptViewDist.jsp?ccode={village_code}&blk={block_code}&dist={district_code}&div={en_division_name}&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_block_name}&seasonName={mr_season_name}"
   pass
elif district_code == '-':
    pass
elif block_code == '-':
    pass
elif village_code == '-':
    pass
else:
    url_params = f"rptViewKhateNew.jsp?ccode={village_code}&blk={block_code}&dist={district_code}&div={en_division_name}&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_block_name}&seasonName={mr_season_name}"
page = requests.get(base_url+url_params)
# print(page)
# html_content = bs(page.content, "html.parser")


In [3]:
html_content = bs(page.content, "html.parser")

In [4]:
object_content = html_content.find('object')
print(object_content)
# sub_url = object_content['data']
# url = base_url+sub_url
# print(url)
# page = requests.get(url)
# html_content=bs(page.content, "html.parser")
print(object_content.find('html'))

<object data="rptViewPeekPahani.jsp?ccode=272100040267930000&amp;blk=4&amp;dist=36&amp;div=kokan&amp;season=खरीप" style="width: 100%; height: 425px;margin: 0 auto;color: #fff; font-size: 15px;"></object>
None


In [73]:
html_result_table =  html_content.find('table')

rows = html_result_table.find_all('tr')

header_row_elements = rows[0].find_all('td')
headers = []
for col in  header_row_elements:
    headers.append(col.text.strip())


AttributeError: 'NoneType' object has no attribute 'find_all'

In [23]:
data_list = []
for i in range(1, len(rows)-2):
    row_elements = rows[i].find_all('td')
    data_elements = []
    for col in row_elements:
        data_elements.append(col.text.strip())
    
    data_list.append(data_elements)

print(data_list)

[['1', 'अकोला', '126520', '193468.64', '0.00', '1712.91', '195181.55'], ['2', 'अमरावती', '228205', '350231.63', '0.00', '4609.60', '354841.22'], ['3', 'बुलडाणा', '232168', '331927.81', '0.00', '2623.36', '334551.16'], ['4', 'यवतमाळ', '225942', '409134.28', '0.00', '6432.11', '415566.38'], ['5', 'वाशिम', '134089', '216559.00', '0.00', '1834.24', '218393.23']]


In [25]:
data_set = pd.DataFrame(data_list, columns=headers)
data_set

,अ.क्र.,जिल्हा,पीक पाहणी खातेदार संख्या,पीक पाहणी क्षेत्र (हे.),कायम पड क्षेत्र (हे.),चालू पड क्षेत्र (हे.),एकूण क्षेत्र (हे.)
0,1,अकोला,126520,193468.64,0.00,1712.91,195181.55
1,2,अमरावती,228205,350231.63,0.00,4609.60,354841.22
2,3,बुलडाणा,232168,331927.81,0.00,2623.36,334551.16
3,4,यवतमाळ,225942,409134.28,0.00,6432.11,415566.38
4,5,वाशिम,134089,216559.00,0.00,1834.24,218393.23


In [26]:
a = b = 2

In [27]:
a, b

(2, 2)

In [91]:
from time import time_ns
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import pandas as pd


In [124]:
base_url = "http://115.124.110.196:8080/epeek/"
# ccode
village_code = "272100040267930000"
# blk
block_code = "4"
# dist
district_code = "36"
# div
en_division_name = "kokan"
# season
mr_season_name = "खरीप"
# divName
mr_division_name = "कोंकण"
# distName
mr_district_name = "पालघर"
# blkName
mr_block_name = "जव्हार"
# ccodeName
mr_village_name = "अनंतनगर"

url_params = ''

if en_division_name == '-':
   url_params =  f"rptViewDiv.jsp?div=pune&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_village_name}&seasonName={mr_season_name}"
elif district_code == '-':
    url_params = f"rptViewDist.jsp?ccode={village_code}&blk={block_code}&dist={district_code}&div={en_division_name}&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_village_name}&seasonName={mr_season_name}"
elif block_code == '-':
    url_params = f"rptViewBlk.jsp?ccode={village_code}&blk={block_code}&dist={district_code}&div={en_division_name}&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_village_name}&seasonName={mr_season_name}"
elif village_code == '-':
    url_params = f"rptViewVill.jsp?ccode={village_code}&blk={block_code}&dist={district_code}&div={en_division_name}&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_village_name}&seasonName={mr_season_name}"
else:
    url_params = f"rptViewKhateNew.jsp?ccode={village_code}&blk={block_code}&dist={district_code}&div={en_division_name}&season={mr_season_name}&divName={mr_division_name}&distName={mr_district_name}&blkName={mr_block_name}&ccodeName={mr_block_name}&seasonName={mr_season_name}"

url = base_url + url_params

In [125]:
# url = "http://115.124.110.196:8080/epeek/rptViewDist.jsp?ccode=-&blk=-&dist=-&div=amravati&season=खरीप&divName=अमरावती&distName=सर्व&blkName=सर्व&ccodeName=सर्व&seasonName=खरीप"
print(url)

http://115.124.110.196:8080/epeek/rptViewKhateNew.jsp?ccode=272100040267930000&blk=4&dist=36&div=kokan&season=खरीप&divName=कोंकण&distName=पालघर&blkName=जव्हार&ccodeName=जव्हार&seasonName=खरीप


In [126]:
browser = webdriver.Firefox()
browser.get(url)
try:
    object_content = browser.find_element(By.TAG_NAME, 'object')
    browser.switch_to.frame(object_content)
except: 
    pass

html_content = bs(browser.page_source, "html.parser")
browser.close()

html_result_table =  html_content.find('table')

rows = html_result_table.find_all('tr')

header_row_elements = rows[0].find_all('td')
headers = []
for col in  header_row_elements:
    headers.append(col.text.strip())



In [127]:
data_list = []
for i in range(1, len(rows)-2):
    row_elements = rows[i].find_all('td')
    data_elements = []
    for col in row_elements:
        data_elements.append(col.text.strip())
    
    data_list.append(data_elements)

In [128]:
data_set = pd.DataFrame(data_list, columns=headers)
data_set.to_csv(f"data/data_{time_ns()}.csv")